In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

data = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')

# #Replacing Null values of Company_type to Other
data['company_type'].fillna('NpO', inplace = True)
test['company_type'].fillna('NpO', inplace = True)

#replacing with medium company size
data['company_size'].fillna('50-99', inplace  = True)
test['company_size'].fillna('50-99', inplace  = True)

# # #Managing all values accordingly
data.experience[data.experience == '>20'] = 20 
data.experience[data.experience == '<1'] = 1 

test.experience[test.experience == '>20'] = 20 
test.experience[test.experience == '<1'] = 1 

# #Replacing NULL values in experience with median
data['experience'].fillna(9, inplace = True)
test['experience'].fillna(9, inplace = True)

# #Replacing all values in major decipline with 'Other
data['major_discipline'].fillna('Other', inplace = True)
test['major_discipline'].fillna('Other', inplace = True)

# #Handling all the values 
data.last_new_job[data.last_new_job == '>4'] = 5 
test.last_new_job[test.last_new_job == '>4'] = 5 

# #Replscing all the NULL values with Mode
data['last_new_job'].fillna(2, inplace = True)
test['last_new_job'].fillna(2, inplace = True)

# #Replacing all the NULL values in education_evel with mode
data['education_level'].fillna('Masters', inplace = True)
test['education_level'].fillna('Masters', inplace = True) 

# #Replacing all the NULL values in enrolled university with mode
data['enrolled_university'].fillna('Part time course', inplace = True)
test['enrolled_university'].fillna('Part time course', inplace = True)

# #Filling NULL values with mode
data.gender.fillna('Other', inplace =  True) #data['gender'].mode()[0]
test.gender.fillna('Other', inplace =  True) #test['gender'].mode()[0]

X = data.drop(['target'] ,axis = 1)
y = data['target'].copy()
X_test = test

from sklearn import preprocessing

for i in X.columns:
    if X[i].dtype == 'object':
        if len(list(X[i].unique())) >=2:
            #cols.append(i)
            le = preprocessing.LabelEncoder()
            le.fit(X[i].astype(str))
            X[i] = le.transform(X[i].astype(str))
            X_test[i] = le.transform(X_test[i].astype(str))
            #X_val[i] =   le.transform(X_val[i].astype(str))
            #X_test[i] =  le.transform(X_test[i].astype(str))
            


X = pd.concat([X.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X['education_level'], prefix='education_level'),
                           pd.get_dummies(X['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X['company_size'], prefix='company_size'),
                           pd.get_dummies(X['company_type'], prefix='company_type')],axis=1)

X_test = pd.concat([X_test.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X_test['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X_test['education_level'], prefix='education_level'),
                           pd.get_dummies(X_test['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X_test['company_size'], prefix='company_size'),
                           pd.get_dummies(X_test['company_type'], prefix='company_type')],axis=1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-p

# tpot

In [2]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
c:\users\admin\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: 0.8684258522601056


Generation 2 - Current best internal CV score: 0.8684258522601056


Generation 3 - Current best internal CV score: 0.8686074320186619


Generation 4 - Current best internal CV score: 0.8686074320186619


Generation 5 - Current best internal CV score: 0.8686074320186619



Best pipeline: LogisticRegression(RobustScaler(input_matrix), C=10.0, dual=False, penalty=l2)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=5, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=20, periodic_checkpoint_folder=None,
        population_size=20, random_state=None, scoring=None, subsample=1.0,
        verbosity=2, warm_start=False)

In [3]:
pred = tpot.predict_proba(X_test)

In [4]:
pred

array([[0.50288712, 0.49711288],
       [0.89431828, 0.10568172],
       [0.77146367, 0.22853633],
       ...,
       [0.94539752, 0.05460248],
       [0.64661663, 0.35338337],
       [0.72162237, 0.27837763]])

In [5]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')

In [7]:
sub['target'] = pred[:,1]

In [8]:
sub.to_csv('tpot.csv',index =False)